In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2023-12-19 12:38:57.921265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 12:38:57.921377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 12:38:57.982362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import shutil

# Source directory containing images
src_dir = './TRAIN'

# Destination directory to copy images
dst_dir = './TEST'

# Check if destination directory exists, create it if it doesn't
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Loop through all files in source directory
for file_name in os.listdir(src_dir):
    # Check if file ends with .jpg
    if len(os.listdir(dst_dir)) == 50:
        break
    if file_name.endswith('.jpg'):
        # Construct full file paths
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        # Copy file to destination directory
        shutil.copy(src_path, dst_path)




In [3]:
from transformers import TFViTForImageClassification

# Replace "your_fine_tuned_model_directory" with the actual directory where your fine-tuned model is stored.
model2 = TFViTForImageClassification.from_pretrained("./plantdoc_ViT_training/vit-base-patch16-224-in21k")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/best.pt').to(device)

# Class mapping
# class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets/train/images'
# directory = './TEST'

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at ./plantdoc_ViT_training/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [4]:
class_mapping = {}

# Assuming your dataset is a pandas DataFrame named 'df' and the column with class labels is 'class'
for i in range(29):
    predicted_class_label = model2.config.id2label[i]
    class_mapping[predicted_class_label] = i

print("Class Mapping:", class_mapping)

Class Mapping: {'Soyabean leaf': 0, 'Tomato leaf late blight': 1, 'Corn Gray leaf spot': 2, 'Cherry leaf': 3, 'Tomato Septoria leaf spot': 4, 'Tomato two spotted spider mites leaf': 5, 'Potato leaf late blight': 6, 'Tomato leaf mosaic virus': 7, 'Corn rust leaf': 8, 'Squash Powdery mildew leaf': 9, 'Bell_pepper leaf': 10, 'Apple leaf': 11, 'Potato leaf early blight': 12, 'grape leaf black rot': 13, 'Strawberry leaf': 14, 'Apple rust leaf': 15, 'Tomato mold leaf': 16, 'Tomato leaf bacterial spot': 17, 'Corn leaf blight': 18, 'Blueberry leaf': 19, 'grape leaf': 20, 'Tomato leaf yellow virus': 21, 'Raspberry leaf': 22, 'Bell_pepper leaf spot': 23, 'Tomato Early blight leaf': 24, 'Potato leaf': 25, 'Peach leaf': 26, 'Tomato leaf': 27, 'Apple Scab Leaf': 28}


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [7]:
from transformers import ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained('./plantdoc_ViT_training/vit-base-patch16-224-in21k')
df = pd.read_csv('./YOLOv8_training/new_labels.csv')
# df = pd.read_csv('./set.csv')

# Define the target size for ResNet50
target_size = (224, 224)
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            inputs = feature_extractor(images=cropped_im)
            pixel_values_tensor = tf.convert_to_tensor(inputs['pixel_values'], dtype=tf.float32)
            outputs = model2(pixel_values=pixel_values_tensor)
            logits = outputs.logits
            logits = tf.nn.softmax(logits)
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = tf.argmax(logits, axis=-1).numpy().item()
            predicted_class_label=model2.config.id2label[predicted_class_index]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([tf.reduce_max(logits).numpy()]).to(device)*r.boxes.conf
#             score_tensor=r.boxes.conf
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    

/home/suryansh/anaconda3/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(



image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2458.jpg: 480x640 12 leafs, 81.7ms
Speed: 2.3ms preprocess, 81.7ms inference, 256.5ms postprocess per image at shape (1, 3, 480, 640)
10   4
10   7
10   6
10   9
10   0
10   8
10   5
10   0
10   2
10   7
10   3
10   8

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img638.jpg: 576x640 1 leaf, 67.5ms
Speed: 3.5ms preprocess, 67.5ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img454.jpg: 544x640 1 leaf, 66.9ms
Speed: 2.2ms preprocess, 66.9ms inference, 1.4ms postprocess per image at shape (1, 3, 544, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1086.jpg: 416x640 2 leafs, 66.6ms
Speed: 2.1ms preprocess, 66.6ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)
2   1
2   0

imag

5   2
5   2
5   3
5   4
5   4
5   0
5   2
5   0
5   2
5   0
5   0
5   0
5   0
5   0
5   1
5   2
5   1
5   2
5   0
5   2
5   0
5   0
5   2
5   2
5   1
5   4
5   1
5   0
5   1
5   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img659.jpg: 512x640 3 leafs, 13.1ms
Speed: 1.7ms preprocess, 13.1ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)
2   1
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1171.jpg: 480x640 19 leafs, 14.7ms
Speed: 2.0ms preprocess, 14.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
9   0
9   0
9   7
9   6
9   2
9   6
9   7
9   4
9   3
9   7
9   1
9   1
9   3
9   5
9   6
9   3
9   0
9   0
9   6

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1538.jpg: 480x640 8 leafs, 14.6ms
Speed: 1.6ms preprocess, 14.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
9   1
9   6
9   5
9   

1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1534.jpg: 640x640 2 leafs, 15.3ms
Speed: 1.9ms preprocess, 15.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img125.jpg: 480x640 1 leaf, 14.6ms
Speed: 1.3ms preprocess, 14.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1584.jpg: 480x640 1 leaf, 14.8ms
Speed: 1.5ms preprocess, 14.8ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img287.jpg: 608x640 43 leafs, 14.3ms
Speed: 2.1ms preprocess, 14.3ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 640)
17   7
17   12
17   12
17   5
17   14
17   14
17   13
17   12
17   10
17   5
17   13


5   2
5   3
5   1
5   0
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2167.jpg: 640x480 5 leafs, 13.9ms
Speed: 1.7ms preprocess, 13.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
1   0
1   0
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1136.jpg: 512x640 1 leaf, 13.6ms
Speed: 1.6ms preprocess, 13.6ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1789.jpg: 640x640 10 leafs, 16.6ms
Speed: 1.9ms preprocess, 16.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
10   8
10   6
10   9
10   2
10   7
10   0
10   3
10   6
10   7
10   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2145.jpg: 640x640 5 leafs, 17.2ms
Speed: 1.9ms preprocess, 17.2ms inference, 1.4ms postprocess per image at

1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1765.jpg: 576x640 14 leafs, 13.4ms
Speed: 2.1ms preprocess, 13.4ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 640)
7   2
7   0
7   5
7   6
7   3
7   3
7   3
7   5
7   1
7   1
7   4
7   0
7   6
7   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1100.jpg: 544x640 5 leafs, 14.0ms
Speed: 1.8ms preprocess, 14.0ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)
5   0
5   2
5   1
5   3
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img135.jpg: 416x640 1 leaf, 11.8ms
Speed: 1.5ms preprocess, 11.8ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img326.jpg: 640x640 2 leafs, 16.3ms
Speed: 1.9ms preprocess, 16.3ms inference, 1.4ms postprocess per image at shape (1, 3

1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1353.jpg: 288x640 2 leafs, 78.1ms
Speed: 1.3ms preprocess, 78.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1636.jpg: 640x480 1 leaf, 13.5ms
Speed: 1.4ms preprocess, 13.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img1568.jpg: 384x640 2 leafs, 11.5ms
Speed: 1.6ms preprocess, 11.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2034.jpg: 448x640 1 leaf, 13.0ms
Speed: 1.7ms preprocess, 13.0ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection

20   1
20   12
20   10
20   10
20   6
20   5
20   1
20   13
20   3
20   10
20   14
20   5
20   9
20   11
20   9
20   5
20   17
20   8
20   12
20   17

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img225.jpg: 640x448 1 leaf, 13.3ms
Speed: 1.6ms preprocess, 13.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 448)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img2496.jpg: 480x640 34 leafs, 14.8ms
Speed: 2.4ms preprocess, 14.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
33   0
33   12
33   24
33   23
33   15
33   29
33   9
33   18
33   9
33   28
33   15
33   10
33   29
33   4
33   19
33   1
33   26
33   6
33   20
33   8
33   18
33   25
33   14
33   5
33   26
33   13
33   16
33   27
33   3
33   12
33   19
33   31
33   8
33   15

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img346.jpg: 544x640 5 leafs, 14.2ms
Speed: 1.8ms pr

Speed: 1.7ms preprocess, 14.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img94.jpg: 480x640 2 leafs, 13.8ms
Speed: 1.6ms preprocess, 13.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img737.jpg: 640x640 4 leafs, 16.5ms
Speed: 2.3ms preprocess, 16.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
3   0
3   1
3   2
3   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img0.jpg: 480x640 3 leafs, 14.8ms
Speed: 1.9ms preprocess, 14.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/test/images/img983.jpg: 640x480 4 leafs, 14.4ms
Speed: 1.9ms preprocess, 14.4ms inference, 1.3ms post

In [8]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [9]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print([preds[i]], [data[i]])
    metrics.update([preds[i]], [data[i]])
print(metrics.compute())

[{'boxes': tensor([[305.9838, 293.8942, 437.3342, 479.8495]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5604], device='cuda:0')}] [{'boxes': tensor([[312, 288, 445, 475]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[ 14.0796, 155.6635, 212.2745, 286.5225]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.3501], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[209.6182, 224.8037, 342.9072, 447.2449]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.4114], device='cuda:0')}] [{'boxes': tensor([[215, 240, 323, 430]], device='cuda:0'), 'labels': tensor([7], device='cuda:0')}]
[{'boxes': tensor([[238.8710,  19.7182, 340.7936, 225.7074]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.5938], device='cuda:0')}] [{'boxes': tensor([[249,  18,

[{'boxes': tensor([[542.1703, 200.8546, 800.0000, 536.8057]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.3270], device='cuda:0')}] [{'boxes': tensor([[609, 239, 774, 430]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}]
[{'boxes': tensor([[249.8591,   0.0000, 439.9714, 162.8380]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.3280], device='cuda:0')}] [{'boxes': tensor([[289,   8, 564, 477]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}]
[{'boxes': tensor([[5.6976e+02, 1.1760e-01, 8.0000e+02, 1.0344e+02]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.2993], device='cuda:0')}] [{'boxes': tensor([[609, 239, 774, 430]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}]
[{'boxes': tensor([[486.9205,  89.6917, 755.4143, 446.0382]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.2634], device='cuda:0')}] [{'boxes': tensor(

[{'boxes': tensor([[550.0726, 409.6967, 588.8032, 456.2343]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.7425], device='cuda:0')}] [{'boxes': tensor([[504, 168, 590, 293]], device='cuda:0'), 'labels': tensor([16], device='cuda:0')}]
[{'boxes': tensor([[699.3642, 396.5309, 735.9363, 468.2104]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.7353], device='cuda:0')}] [{'boxes': tensor([[504, 168, 590, 293]], device='cuda:0'), 'labels': tensor([16], device='cuda:0')}]
[{'boxes': tensor([[ 37.1904, 110.9505, 154.8009, 177.9732]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.7207], device='cuda:0')}] [{'boxes': tensor([[ 38, 102, 163, 190]], device='cuda:0'), 'labels': tensor([16], device='cuda:0')}]
[{'boxes': tensor([[246.2909, 513.8486, 291.9652, 567.3459]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.7199], device='cuda:0')}] [{'boxes': tensor([[165, 1

[{'boxes': tensor([[2833.6091, 2109.1941, 4056.4724, 3238.2024]], device='cuda:0'), 'labels': tensor([9], device='cuda:0'), 'scores': tensor([0.5282], device='cuda:0')}] [{'boxes': tensor([[2981, 2072, 4060, 3240]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}]
[{'boxes': tensor([[  79.3430, 2474.4016, 1318.0598, 3239.7166]], device='cuda:0'), 'labels': tensor([9], device='cuda:0'), 'scores': tensor([0.3524], device='cuda:0')}] [{'boxes': tensor([[1564, 1231, 2931, 3240]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}]
[{'boxes': tensor([[5.2195e+02, 3.2852e-01, 7.4200e+02, 3.0277e+02]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.3411], device='cuda:0')}] [{'boxes': tensor([[523,   6, 742, 286]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}]
[{'boxes': tensor([[ 24.2101,   4.2984, 535.9792, 469.1835]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.2500], device='cuda:0')}] [{'box

[{'boxes': tensor([[1543.8799, 2727.5566, 3000.0000, 3998.7441]], device='cuda:0'), 'labels': tensor([9], device='cuda:0'), 'scores': tensor([0.2725], device='cuda:0')}] [{'boxes': tensor([[1641, 3021, 3000, 3937]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}]
[{'boxes': tensor([[ 42.5925,  42.1165, 370.0552, 493.4657]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.5148], device='cuda:0')}] [{'boxes': tensor([[ 40,  41, 369, 484]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}]
[{'boxes': tensor([[1.0741e+02, 1.8531e-01, 2.7224e+02, 1.0355e+02]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.4215], device='cuda:0')}] [{'boxes': tensor([[ 40,  41, 369, 484]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}]
[{'boxes': tensor([[2.4189e+02, 8.8882e-02, 3.7500e+02, 3.0869e+02]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.1367], device='cuda:0')}] [{'box

...

[{'boxes': tensor([[479.4953, 594.9759, 616.2879, 749.9085]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.2538], device='cuda:0')}] [{'boxes': tensor([[478, 584, 601, 750]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[767.4898, 514.9230, 954.8920, 650.7233]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.2297], device='cuda:0')}] [{'boxes': tensor([[747, 478, 958, 653]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[  0.0000, 319.9024,  61.7315, 524.5056]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.2233], device='cuda:0')}] [{'boxes': tensor([[ 47, 315, 381, 601]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}]
[{'boxes': tensor([[114.6610, 185.6346, 263.2474, 422.7440]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.1312], device='cuda:0')}] [{'boxes': tensor([[108, 2

[{'boxes': tensor([[722.9012, 351.9665, 874.3846, 482.6025]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.6827], device='cuda:0')}] [{'boxes': tensor([[726, 358, 854, 483]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[205.5279, 384.4888, 532.8635, 519.8307]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.7538], device='cuda:0')}] [{'boxes': tensor([[178, 405, 533, 525]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[584.5871,  53.5552, 741.5161, 350.5012]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.7020], device='cuda:0')}] [{'boxes': tensor([[583,  71, 730, 339]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[ 49.2860, 424.2109, 158.5846, 481.3078]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.6325], device='cuda:0')}] [{'boxes': tensor([[ 52, 4

[{'boxes': tensor([[336.2901, 125.1613, 380.6089, 193.2384]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.0812], device='cuda:0')}] [{'boxes': tensor([[329, 118, 382, 192]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[ 36.5079, 145.9579,  68.9674, 222.2425]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.1101], device='cuda:0')}] [{'boxes': tensor([[ 34, 144,  64, 226]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[363.9920, 100.5798, 404.1138, 147.0986]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.0928], device='cuda:0')}] [{'boxes': tensor([[366, 101, 405, 153]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[435.6350, 192.9213, 465.6646, 260.0291]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.0991], device='cuda:0')}] [{'boxes': tensor([[439, 1

[{'boxes': tensor([[230.7841,  87.2870, 255.3781, 123.9946]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.0622], device='cuda:0')}] [{'boxes': tensor([[227,  89, 256, 125]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[250.7891, 305.8417, 277.3649, 344.5120]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.0624], device='cuda:0')}] [{'boxes': tensor([[252, 307, 272, 347]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[310.7556,   0.8211, 330.5138,  54.6949]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.0587], device='cuda:0')}] [{'boxes': tensor([[335, 106, 355, 161]], device='cuda:0'), 'labels': tensor([26], device='cuda:0')}]
[{'boxes': tensor([[244.6917, 190.6176, 263.1254, 221.9515]], device='cuda:0'), 'labels': tensor([26], device='cuda:0'), 'scores': tensor([0.0462], device='cuda:0')}] [{'boxes': tensor([[226, 1

[{'boxes': tensor([[ 57.0316,  14.3671, 829.2164, 763.2884]], device='cuda:0'), 'labels': tensor([9], device='cuda:0'), 'scores': tensor([0.7582], device='cuda:0')}] [{'boxes': tensor([[ 62,  14, 829, 776]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}]
[{'boxes': tensor([[ 126.2545,  252.1815, 5144.1694, 3436.7844]], device='cuda:0'), 'labels': tensor([18], device='cuda:0'), 'scores': tensor([0.7463], device='cuda:0')}] [{'boxes': tensor([[ 110,  268, 5133, 3427]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
{'map': tensor(0.2413), 'map_50': tensor(0.3368), 'map_75': tensor(0.2665), 'map_small': tensor(0.1616), 'map_medium': tensor(0.1655), 'map_large': tensor(0.2533), 'mar_1': tensor(0.3033), 'mar_10': tensor(0.3033), 'mar_100': tensor(0.3033), 'mar_small': tensor(0.1856), 'mar_medium': tensor(0.2038), 'mar_large': tensor(0.3162), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 